# Arroyo SONAs

This processes all collated Arroyo SONA. Reminder to run the [Philippines SONA](https://github.com/pmagtulis/ph-sona.git) scraper file to collect the **merged** CSV file here.

## Do all your imports

In [11]:
import pandas as pd
import numpy as np
import re
import altair as alt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import stopwordsiso as stopwords

## Read CSV

In [12]:
merged= pd.read_csv('../csv/merged.csv')
merged

Unnamed: 0                president               date  \
0            0         Manuel L. Quezon  November 25, 1935   
1            1         Manuel L. Quezon      June 16, 1936   
2            2         Manuel L. Quezon   October 18, 1937   
3            3         Manuel L. Quezon   January 24, 1938   
4            4         Manuel L. Quezon   January 24, 1939   
..         ...                      ...                ...   
79          79      Rodrigo Roa Duterte      July 23, 2018   
80          80      Rodrigo Roa Duterte      July 22, 2019   
81          81      Rodrigo Roa Duterte      July 27, 2020   
82          82      Rodrigo Roa Duterte      July 26, 2021   
83          83  Ferdinand R. Marcos Jr.      July 25, 2022   

                                                                                                           title  \
0                                                              Message to the First Assembly on National Defense   
1                                                                       On the Country’s Conditions and Problems   
2   Improvement of Philippine Conditions, Philippine Independence, and Relations with American High Commissioner   
3                                                                             Revision of the System of Taxation   
4                                                        The State of the Nation and Important Economic Problems   
..                                                                                                           ...   
79                                                                             Third State of the Nation Address   
80                                                                            Fourth State of the Nation Address   
81                                                                             Fifth State of the Nation Address   
82                                                                             Sixth State of the Nation Address   
83                                                                             First State of the Nation Address   

                                                                                                                                    link  \
0   http://www.officialgazette.gov.ph/1935/11/25/message-of-president-quezon-to-the-first-assembly-on-national-defense-november-25-1935/   
1                          http://www.officialgazette.gov.ph/1936/06/16/manuel-l-quezon-second-state-of-the-nation-address-june-16-1936/   
2                        http://www.officialgazette.gov.ph/1937/10/18/manuel-l-quezon-third-state-of-the-nation-address-october-18-1937/   
3                       http://www.officialgazette.gov.ph/1938/01/24/manuel-l-quezon-fourth-state-of-the-nation-address-january-24-1938/   
4                        http://www.officialgazette.gov.ph/1939/01/24/manuel-l-quezon-fifth-state-of-the-nation-address-january-24-1939/   
..                                                                                                                                   ...   
79                     https://www.officialgazette.gov.ph/2018/07/23/rodrigo-roa-duterte-third-state-of-the-nation-address-july-23-2018/   
80                    https://www.officialgazette.gov.ph/2019/07/22/rodrigo-roa-duterte-fourth-state-of-the-nation-address-july-22-2019/   
81                     https://www.officialgazette.gov.ph/2020/07/27/rodrigo-roa-duterte-fifth-state-of-the-nation-address-july-27-2020/   
82                     https://www.officialgazette.gov.ph/2021/07/26/rodrigo-roa-duterte-sixth-state-of-the-nation-address-july-26-2021/   
83                https://www.officialgazette.gov.ph/2021/07/26/ferdinand-r-marcos-jr-first-state-of-the-nation-address-july-25-2022/%22   

                             venue                                  session  \
0     Legislative Building, Manila   First National Assembly, First Session   
1     Legislative Building,

# Initial analysis

## regex

We are now ready to take an **initial analysis** of the texts that we have. For this part, I provided some examples below using **regex**.

An important note on this method: the **str.contains** and **str.extractall** functions **ONLY** count *the number of speeches* that contain the word, *not how many times* the word was mentioned in the speech. We would look into the count of the words on the speeches later at a deeper analysis.

Words we ran here are based from peer-reviewed textual studies that gauge **populism.**

### 'elite'

The word "elite" is found to have been often used by populist leaders. We find based on this initial analysis that in the case of Philippine presidents, three leaders (one of whom was **dictator** Ferdinand Marcos Sr.) were found to have included the word in their SONAs.

In [13]:
merged[merged.speech.str.contains(r"\belite", case=False, regex=True)].president.value_counts()

Ferdinand E. Marcos        2
Joseph Ejercito Estrada    1
Rodrigo Roa Duterte        1
Name: president, dtype: int64

In [14]:
pd.set_option('display.max_colwidth', None)
merged.speech.str.extractall(r'(.*\belite.+)', re.IGNORECASE)

,,0
,match,
31,0,"It is fortunate that the nation will, just two years from now, call a constitutional convention. I leave it to the delegates of that convention to evolve a truly democratic system, one which will not merely bend, as our system does today, to the wishes of a traditional elite and perpetuate the status quo. Democratic institutions must be instruments of national advancement. Democracy must symbolize change."
37,0,"Clearly, we face here the danger that our New Society is giving birth to a new government elite, who resurrect in our midst the privileges we fought in the past, who employ the powers of high office for their personal enrichment, as well as of their business colleagues, relatives, and friends."
60,0,"Our war on poverty is in the acceleration of the land redistribution processes under the agrarian reform program. We distributed more than 266,000 hectares of land to 175,000 landless farmers, including land owned by the traditional rural elite. []"
81,0,Great wealth enables economic elites and corporations to influence public policy to their advantage. Media is a powerful tool in the hands of oligarchs like the Lopezes who used their media outlets in their battles with political figures. I am a casualty of the Lopezes during the 2016 election.


### 'democracy' and 'demokrasya'

Dictator Ferdinand E. Marcos mentioned the word **"democracy"** in 10 of his SONAs followed by Gloria Arroyo (7 of 9 SONAs). In Filipino, Benigno Aquino III mentioned **"demokrasya"** in two of his six speeches. 



**Joseph Estrada**, whose term was cut short by a popular revolt in 2001, and **Rodrigo Duterte** mentioned the word in a single SONA. 

In [15]:
merged[merged.speech.str.contains(r"(.*\bdemocracy.+)", case=False, regex=True)].president.value_counts()

/var/folders/00/cqb74mkn60g_hf2zk42fy8_m0000gp/T/ipykernel_2678/3410026565.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  merged[merged.speech.str.contains(r"(.*\bdemocracy.+)", case=False, regex=True)].president.value_counts()


Ferdinand E. Marcos        10
Gloria Macapagal-Arroyo     7
Manuel L. Quezon            5
Corazon C. Aquino           5
Fidel V. Ramos              5
Ramon Magsaysay             4
Diosdado Macapagal          4
Manuel Roxas                3
Elpidio Quirino             3
Carlos P. Garcia            2
Joseph Ejercito Estrada     1
Rodrigo Roa Duterte         1
Name: president, dtype: int64

In [16]:
merged[merged.speech.str.contains(r"(.*\bdemokrasya.+)", case=False, regex=True)].president.value_counts()

/var/folders/00/cqb74mkn60g_hf2zk42fy8_m0000gp/T/ipykernel_2678/1026723483.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  merged[merged.speech.str.contains(r"(.*\bdemokrasya.+)", case=False, regex=True)].president.value_counts()


Benigno S. Aquino III      2
Ferdinand E. Marcos        1
Corazon C. Aquino          1
Gloria Macapagal-Arroyo    1
Name: president, dtype: int64

In [17]:
merged.speech.str.extractall(r'(.*\bdemocracy.+)', re.IGNORECASE).head(7)

0
  match                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
1 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        In our day and generation democracy, as an effective system of government, is being challenged. Let this new democracy of ours show to the world that democracy can be as efficient as a dictatorship, without trespassing upon individual liberty and the sacred rights of the people.
2 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Still more: The Filipino workingman has heard, if he is not able to read, of the equality before the law of the poor and the rich. He has heard of democracy, liberty, and justice, since every candidate for an elective office discourses on these topics, painting to him in glowing terms the meaning of these words.
  1                                                                                                                                                                                                                                                                                                             One of the discoveries which we have made since the establishment of the Government of the Commonwealth is that, despite the large number of children that have gone through our public schools, as shown in the reports of the Bureau of Education, the literacy of the Islands has not increased proportionally, and the knowledge of those rudimentary subjects which the citizen of a democracy should have, has not been acquired by a population corresponding to the number of children that appear to have entered the public schools. The reason for this is simple. A large proprtion of the boys and girls who have been admitted to the schools have not remained long en

In [18]:
merged.speech.str.extractall(r'(.*\bdemokrasya.+)', re.IGNORECASE).head()

0
   match                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
40 0                            Nasa harap ng kapulungang ito ngayon ang katipunan ng mga hamon at pagsubok sa nakalipas na mga Kongreso, at ito na sana ang pangwakas na pagsubok kung makakaya natin gamitin ang demokrasya bilang mabisang sangkap ng katatagan at kaunlarang pambansa. Bagaman at kailangan pa ring magpatuloy ang pansamantalang pamahalaan, taglay ng kapulungang ito ang binhi ng matatag at masiglang lehislaturang tutugon sa ating pangangailangan kung ihahandog natin dito ang lahat ng ating talino at kakayahan.
   1      Tayo ngayon ay isang bansang pinalakas ng mga pagsubok na ating pinagdaanan, higit na nagkakaisa pagkaraan ng mga sigalutang dinanas, at higit na handa sa anumang uri ng pagsubok at suliranin. Natapos nating lampasan ang mahihigpit na balakid sa nakaraang lima-at-kalahating taon. Sa liwanag ng makabuluhang yugtong ito ng ating buhay bilang bansa at lahi, magagawa natin ang ating tungkuling pagtahak sa landas ng katuparan ng ating matayog na pangarap na pag-unlad, pagkakapantay-pantay, at ng tunay na demokrasya.
51 0                                                                                                                                                                                                                                                                                                                                                                                                Binigyang buhay ng mga Kabisig nating ito ang diwa ng ating Saligang Batas; binigyang halimbawa nila ang tunay na kahulugan ng demokrasya.
   1                                                                                                                                                                                May katiyakan ang ating tagumpay kung tayo’y magkakaisa. Kung kaya’t hinihimok ko kayo—kagalang-galang na mga Senador, Kongresista, at ang iba pang mga pinuno ng bayan—na muli tayong manumpa sa pangarap na nagbigkis sa atin noong 1986: ibalik at panatiliin ang demokrasya, kalayaan, karapatan, katatagang pangkabuhayan, at katarungang panlipunan.
65 0                                                                                                                                                                                                                                            Pinapangako ko ang isang bagong direksyon: mamamayan muna. Ang taong bayan ang pinakamalaki nating yaman. Ngunit madalas, kaunti lang ang atensyon na binibigay sa kanilang pag-unlad. Di tuloy matawid ang agwat ng mayaman at mahirap. Di tuloy mapa-abot sa lahat ang biyaya ng demokrasya.

## Segregating by president

We create separate dataframes from a select number of presidents to analyze using text analysis.

In [19]:
#Post-martial law
cory = merged[(merged['president'] == 'Corazon C. Aquino')] #Cory Aquino
ramos = merged[(merged['president'] == 'Fidel V. Ramos')] #Fidel Ramos
aquino = merged[(merged['president'] == 'Benigno S. Aquino III')] #Aquino
duterte = merged[(merged['president'] == 'Rodrigo Roa Duterte')] #Duterte
erap = merged[(merged['president'] == 'Joseph Ejercito Estrada')] #Erap
arroyo = merged[(merged['president'] == 'Gloria Macapagal-Arroyo')] #Arroyo
marcosjr = merged[(merged['president'] == 'Ferdinand R. Marcos Jr.')] #Marcos Jr.

marcos = merged[(merged['president'] == 'Ferdinand E. Marcos')] #Marcos Sr.

# Pre-martial law
macapagal = merged[(merged['president'] == 'Diosdado Macapagal')] #Diosdado Macapagal
garcia = merged[(merged['president'] == 'Carlos P. Garcia')] #Carlos Garcia
magsaysay = merged[(merged['president'] == 'Ramon Magsaysay')] #Ramon Magsaysay
quirino = merged[(merged['president'] == 'Elpidio Quirino')] #Elpidio Quirino

## Isolate 'Arroyo' speeches

The merged file contains all speeches by Philippine presidents since 1935. 

In [20]:
arroyo = merged[(merged['president'] == 'Gloria Macapagal-Arroyo')] #Erap

## Text analysis

Now, we can proceed with the text analysis proper. First stop, we set the parameters in the immediate cell below, most importantly the stopwords we want our analysis to disregard.

In [21]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    return text #removes all numbers

In [84]:
y_columns = ['president', 'speeches']
BINARY=False
NGRAM_RANGE=(1,1)
MIN_DF=0
STPWORDS=stopwords.stopwords(["en", 'tl']) #removes Tagalog stopwords
STPWORDS.update(['yung', 'iyan', 'yan', 'diyan', 'applause', 'laughter', 'palakpakan', 'rin', 'din', 'po',
                'pong', 'pang', 'pa', 'nang', 'ng', 'pag',
                'kapag', 'nga', 'upang','naman', 'natin', 'kayo',
                'nating', 'natin', 'tayong', 'lang', 'jayson', 'jomar', 'erwin']) #adds more Tagalog stopwords not included in the package 

vectorizer = CountVectorizer(
    stop_words=STPWORDS,
    ngram_range=NGRAM_RANGE,
    binary=BINARY,
    min_df=MIN_DF,
    preprocessor=preprocess_text
)

## Vectorizing

Simple counting of words that occur in a speech.

In [85]:
X = vectorizer.fit_transform(arroyo['speech'])
X

/opt/homebrew/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


<9x5916 sparse matrix of type '<class 'numpy.int64'>'
	with 10443 stored elements in Compressed Sparse Row format>

In [86]:
arroyo_vectors = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# [print(x) for x in marcosjr.speech]
arroyo_vectors.round(2)

,aabala,aalala,aalisin,aaral,aasikaso,aatras,aatubiling,abandon,abandoned,abated,...,youth,youtube,yugto,yumayabong,yun,yzfmmjwa_no,zambales,zamboanga,zone,zubiri
0,0,0,0,2,1,0,0,0,1,0,...,3,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,1,3,0,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,3,3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,1,3,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,1,0,2,0,0,0,0,2,2,2
6,0,0,0,2,0,0,0,0,0,0,...,1,3,0,0,1,0,0,0,1,1
7,1,4,0,0,0,0,0,0,0,0,...,1,3,0,0,0,1,0,4,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
arroyo_vectors = arroyo_vectors.transpose() #swapping columns and row positions

In [88]:
arroyo_vectors.columns = ['SONA1', 'SONA2', 'SONA3', 'SONA4', 'SONA5', 'SONA6', 'SONA7', 'SONA8', 'SONA9']
arroyo_vectors.sort_values('SONA1', ascending=False).head(20)

,SONA1,SONA2,SONA3,SONA4,SONA5,SONA6,SONA7,SONA8,SONA9
national,16,7,2,3,4,3,6,4,2
law,15,5,4,2,1,2,4,0,10
trabaho,13,3,5,2,0,3,6,1,1
congress,13,6,10,14,6,4,15,7,9
poverty,11,5,2,0,0,3,1,1,1
economic,11,6,0,5,4,2,2,2,6
taon,11,7,3,0,0,2,6,3,7
jobs,10,3,5,3,1,1,2,2,2
ninyo,9,0,0,0,1,0,0,1,1
bayan,9,0,0,3,2,2,2,8,4


## Add a 'total' mention column

Totally optional, just in case you wanted to find the total number of mentions.

In [89]:
arroyo_vectors['total'] = arroyo_vectors.SONA1 + arroyo_vectors.SONA2 + arroyo_vectors.SONA3 + arroyo_vectors.SONA4 + arroyo_vectors.SONA5 + arroyo_vectors.SONA6 + arroyo_vectors.SONA7 + arroyo_vectors.SONA8 + arroyo_vectors.SONA9   


In [90]:
arroyo_vectors = arroyo_vectors.sort_values('total', ascending=False)
arroyo_vectors.head(15)

,SONA1,SONA2,SONA3,SONA4,SONA5,SONA6,SONA7,SONA8,SONA9,total
government,8,8,6,18,12,9,9,10,7,87
congress,13,6,10,14,6,4,15,7,9,84
people,9,8,2,11,9,12,9,11,12,83
president,8,7,7,10,6,6,6,7,13,70
country,9,7,4,7,9,11,1,7,12,67
nation,9,11,7,5,4,10,3,9,6,64
power,7,11,4,2,2,8,8,3,4,49
national,16,7,2,3,4,3,6,4,2,47
strong,0,20,4,3,4,1,4,2,7,45
mindanao,5,0,3,0,2,9,14,7,4,44


# TF-IDF

## Arroyo speeches

In [91]:
vectorizer = TfidfVectorizer(
    stop_words=STPWORDS, 
    ngram_range=NGRAM_RANGE,
    binary=BINARY,
    min_df=MIN_DF,
    preprocessor=preprocess_text
)
X = vectorizer.fit_transform(arroyo['speech'])
arroyo_idf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
#[print(x) for x in speeches.sentence]
arroyo_idf.round(2)

/opt/homebrew/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


,aabala,aalala,aalisin,aaral,aasikaso,aatras,aatubiling,abandon,abandoned,abated,...,youth,youtube,yugto,yumayabong,yun,yzfmmjwa_no,zambales,zamboanga,zone,zubiri
0,0.00,0.00,0.00,0.03,0.02,0.00,0.00,0.00,0.02,0.00,...,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.01,0.03,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00
2,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.03,0.00,0.00,...,0.01,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.04,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,...,0.02,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,...,0.01,0.00,0.04,0.00,0.00,0.00,0.00,0.03,0.03,0.03
6,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,...,0.01,0.03,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.02
7,0.02,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.01,0.04,0.00,0.00,0.00,0.02,0.00,0.07,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [92]:
arroyo_idf2 = arroyo_idf.transpose()
arroyo_idf2.columns = ['SONA1', 'SONA2', 'SONA3', 'SONA4', 'SONA5', 'SONA6', 'SONA7', 'SONA8', 'SONA9']

In [93]:
arroyo_idf2.sort_values('SONA7', ascending=False).head(15)

,SONA1,SONA2,SONA3,SONA4,SONA5,SONA6,SONA7,SONA8,SONA9
airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.163587,0.245625,0.015180,0.000000
road,0.000000,0.011415,0.015817,0.000000,0.000000,0.128974,0.161378,0.000000,0.013443
mindanao,0.042201,0.000000,0.038415,0.000000,0.040657,0.085431,0.121939,0.067824,0.043535
park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111490,0.000000,0.000000
plant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.109861,0.034918,0.000000
construction,0.026446,0.000000,0.000000,0.000000,0.031848,0.000000,0.109166,0.000000,0.000000
mayor,0.000000,0.000000,0.000000,0.000000,0.000000,0.014872,0.109166,0.015180,0.000000
governor,0.000000,0.000000,0.000000,0.000000,0.000000,0.044615,0.109166,0.000000,0.017051
agribusiness,0.011683,0.000000,0.000000,0.020354,0.000000,0.052559,0.108511,0.000000,0.000000
congress,0.089705,0.045334,0.104690,0.168296,0.099718,0.031042,0.106814,0.055451,0.080083


## Looking for specific words

In this part, we are looking for specific words that we think made a mark during Aquino SONAs, whether because they are often mentioned, or because it is unusual for the Chief Executive to say it. 

We also include here words that we think were said because they were the topic at hand at the time the speech was delivered.

In [94]:
arroyo_slice = arroyo_idf[['mahirap', 'government']] # you can change this
arroyo_slice.sort_index().round(decimals=2)

,mahirap,government
0,0.00,0.06
1,0.00,0.06
2,0.02,0.06
3,0.02,0.22
4,0.00,0.20
5,0.00,0.07
6,0.00,0.06
7,0.00,0.08
8,0.02,0.06


In [95]:
arroyo_slice = arroyo_slice.stack().reset_index()
arroyo_slice = arroyo_slice.rename(columns={'level_0': 'sona_no','level_1': 'term', 'tfidf': 'term', 0: 'tfidf'})
arroyo_slice.head()

,sona_no,term,tfidf
0,0,mahirap,0.000000
1,0,government,0.055203
2,1,mahirap,0.000000
3,1,government,0.060446
4,2,mahirap,0.020062


In [96]:
top_tfidf = arroyo_slice.sort_values(by=['sona_no','tfidf'], ascending=[True,False]).groupby(['sona_no']).head(10)
top_tfidf.head()

,sona_no,term,tfidf
1,0,government,0.055203
0,0,mahirap,0.000000
3,1,government,0.060446
2,1,mahirap,0.000000
5,2,government,0.062814


## Chart it

In [97]:
# # Terms in this list will get a red dot in the visualization
term_list = ['mahirap', 'wangwang'] # you can change this

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'sona_no:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["sona_no"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600, height=400)

alt.LayerChart(...)

## Entire SONAs

In here, we do the same thing for all of SONA *without* isolating key words.

In [98]:
arroyo_idf = arroyo_idf.stack().reset_index()
arroyo_idf

,level_0,level_1,0
0,0,aabala,0.000000
1,0,aalala,0.000000
2,0,aalisin,0.000000
3,0,aaral,0.026446
4,0,aasikaso,0.018006
...,...,...,...
53239,8,yzfmmjwa_no,0.000000
53240,8,zambales,0.000000
53241,8,zamboanga,0.000000
53242,8,zone,0.000000


In [99]:
arroyo_idf = arroyo_idf.rename(columns={'level_0': 'sona_no','level_1': 'term', 0: 'tfidf'})
arroyo_idf

,sona_no,term,tfidf
0,0,aabala,0.000000
1,0,aalala,0.000000
2,0,aalisin,0.000000
3,0,aaral,0.026446
4,0,aasikaso,0.018006
...,...,...,...
53239,8,yzfmmjwa_no,0.000000
53240,8,zambales,0.000000
53241,8,zamboanga,0.000000
53242,8,zone,0.000000


In [100]:
all_arroyo = arroyo_idf.sort_values(by=['sona_no','tfidf'], ascending=[True,False]).groupby(['sona_no']).head(10)
all_arroyo.head()

,sona_no,term,tfidf
2818,0,law,0.114412
3622,0,national,0.110407
3683,0,ninyo,0.105151
5524,0,trabaho,0.099157
5332,0,tahanan,0.093467


In [101]:
# # Terms in this list will get a red dot in the visualization
term_list = ['boss', 'wangwang']

# adding a little randomness to break ties in term ranking
all_arroyo_plusRand = all_arroyo.copy()
all_arroyo_plusRand['tfidf'] = all_arroyo_plusRand['tfidf'] + np.random.rand(all_arroyo.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(all_arroyo_plusRand).encode(
    x = 'rank:O',
    y = 'sona_no:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["sona_no"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600, height=400)

alt.LayerChart(...)